### Lab_7_Inclass_Assignment
- Name: Aagnay Kariyal
- Student ID: 8830232


##### Data Integration: Approximate String Matching
1- Implement FROM SCRATCH the approximate string matching functionality using Levenshtein Distance
- Levenshtein Distance between two strings a,b of length |a| and |b| is calculated by creating a matrix of dimensions |a+1|x|b+1| and then filling the matrix by the following equation:<br>
lev<sub>a,b</sub>(i,j) = <br>
a. max(i,j) IF min(i,j)=0<br>
b. minimum of:    
    <ol> <li>lev<sub>a,b</sub>(i-1,j)+1</li>
    <li>lev<sub>a,b</sub>(i,j-1)+1</li>
    <li>lev<sub>a,b</sub>(i-1,j-1)+ 
            *1 if (a<sub>i</sub> != b<sub>j</sub>), 0 otherwise*</li> </ol>

In [188]:
# Importing Dependencies
import pandas as pd
import numpy as np
import re

In [189]:
def calculate_levenshtein_distance(string_a, string_b): # Creating a function to get the levenshtein distance
    ## TODO: Write the functionality that calculates how many edit operations are required to change one string into another
    ## Example: Distance "HONDA" to "HYUNDAI" is 3, where the operations are adding "Y" after H in Honda
    ## Then replacing O with Y, then keeping N, keeping D, keeping A, adding I.
    
    # Adding 1 to the length of the string to have enough cells in the matrices
    length_a = len(string_a) + 1 
    length_b = len(string_b) + 1
    matrix = np.zeros([length_a, length_b]) # Creating a matrix
    
    # Using a for loop to impute values in their location
    for x in range(length_a):
        matrix[x][0] = x
    for y in range(length_b):
        matrix[0][y] = y
        
    # Using a for loop to iterate through every cell and implement the levenshtein equations    
    for x in range(1, length_a):
        for y in range(1, length_b):
            deletion = matrix[x-1][y] + 1 # Deletion equation with the matrix
            insertion = matrix[x][y-1] + 1 # Insertion equation with the matrix
            if string_a[x-1] == string_b[y-1]: # Substitution conditional check
                subs = 0
            else:
                subs = 1
            substitution = matrix[x-1][y-1] + subs # Inserting the substitution value
            
            matrix[x][y] = min(deletion, insertion, substitution) # Inserting the minimum value out of the three processes into the bottom right cell of the matrix
    return int(matrix[length_a-1, length_b-1]) # Returning the value at the bottom right cell of the matrix after convering it into an int

2- Using the implemented Levenshtein's distance, merge the following dataframes based on the closest match for the "Name" attribute. (You will need to find first the matching pairs)

In [190]:
# First Dataframe
data1 = {'ID1': [1, 2, 3, 4],
         'Name1': ['John Smith', 'Alice Johnson', 'Bob Marley', 'David Doe']}
df1 = pd.DataFrame(data1)
# Second Dataframe
data2 = {'ID2': [101, 102, 103, 104,105],
         'Name2': ['Jon Smith', 'Alyce Jonson', 'Robert Marley', 'Dave D.', 'Rob Marly']}
df2 = pd.DataFrame(data2)

In [191]:
def closest_match(name, list): # Defining a function to get the closest match
    distances = {} # Creating a dictionary to insert the name and levenshtein distance value
    for nme in list: # Iterating through a list of names from the second dataframe
        dist = calculate_levenshtein_distance(name, nme) # Calculating the distance between the two names
        distances.update({nme:dist}) # Updating the dataframe with the name and the distance
    min_distance = min(distances.values()) # Getting the minimum distance value from the dictionary
    pos = '' # Empty string to store the name consisting the minimum distance
    for key, value in distances.items(): # Iterating through the dictionary to find the name
        if value == min_distance: # Conditional statement
            pos = key # Inserting the value of the name from the dict into the empty string
    return name, pos, min_distance # Return the name from df1 and df 2 along with the min distance

In [192]:
levenshtein_df = pd.DataFrame(columns=['n1','n2','distance']) # Creating an empty dataframe with 3 columns to store values
for name in data1['Name1']: # Iterating names through a list of names
    name1, name2, min_dist = closest_match(name, data2['Name2']) # Storing values into variables
    print(f'The closest match for {name1} is {name2} with the Levenshtein distance of {min_dist}') # Printing a statement to check values
    levenshtein_df.loc[len(levenshtein_df)] = [name1,name2,min_dist] # Storing the values into the empty dataframe
    
levenshtein_df # Checking the dataframe

The closest match for John Smith is Jon Smith with the Levenshtein distance of 1
The closest match for Alice Johnson is Alyce Jonson with the Levenshtein distance of 2
The closest match for Bob Marley is Rob Marly with the Levenshtein distance of 2
The closest match for David Doe is Dave D. with the Levenshtein distance of 4


,n1,n2,distance
0,John Smith,Jon Smith,1
1,Alice Johnson,Alyce Jonson,2
2,Bob Marley,Rob Marly,2
3,David Doe,Dave D.,4


In [193]:
id_cols = [] # Empty List to store the id numbers of the name2 from df2
for n in levenshtein_df['n2']: # Iterating names through the new dataframe
    for n2 in df2['Name2']: # Iterating names through df2
        if n == n2: # Checking if both of the names match
            ind = df2[df2['Name2']==n].index.values[0] # Getting the index value of the name from df2
            ids = df2.iloc[ind]['ID2'] # Getting the id by using the index value
            id_cols.append(ids) # Appending the id into the empty list
        
df1['closest_match'] = levenshtein_df['n2'] # Inserting and merging the names from df2 into df1
df1['Id2'] = id_cols # Inserting and merging IDs from df2 into df1
df1       # Checking the values

,ID1,Name1,closest_match,Id2
0,1,John Smith,Jon Smith,101
1,2,Alice Johnson,Alyce Jonson,102
2,3,Bob Marley,Rob Marly,105
3,4,David Doe,Dave D.,104


3- Using bookings and yachts csv:
<ol>
    <li> Inspect the bookings and yachts csv files.</li>
    <li> Before merging, make sure to keep only bookings with correctly formatted Yacht_IDs, where the Yacht_ID should be in a format of two characters, followed by 3 numbers.</li>
    <li> Combine both dataframes into one dataframe.</li>
    <li> Make sure that all columns are of correct format where it is expected that: <br>
        a. Title, First_Name, Last_Name, Street_Address, Area, Post_Code, Telephone, Yacht_ID, Yacht_Name, Yacht_Make, Yacht_Ref, Depart_Port, Depart_Country<br>
        b. Cust_ID, Built, Length(m), Cabins, Toilets, Guiests, Day_Rate are Numbers <br>
        c. Depart_Date is Date (its original format id dd/mm/yyyy)</li>
    
</ol>

***Note: Data may contain Latin Characters, You may need to use Latin Encoding, drop data where name contains any characters other than normal alphabetics***

In [194]:
# Loading the data into dataframes
yacht_df = pd.read_csv('yachts.csv')
bookings_df = pd.read_csv('bookings.csv', encoding='latin-1')
yacht_df # Checking the dataframe

,Yacht_Name,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,Day_Rate
0,Adriatic Star,Balandro Calypso 45,SY001,Pavao Island Marina,Croatia,2013,13.85,4,2,10,656
1,Amber Sunrise,Benedetti 37 Mirage,SY002,Pavao Island Marina,Croatia,2014,11.30,3,1,8,431
2,Anne Elizabeth,Marchal Grande 410,SY003,Port Djurak,Croatia,2016,12.35,3,2,8,486
3,Aqua Calma,Benedetti 37 Mirage,SY004,Marina Naupotiri,Greece,2007,11.35,3,1,8,377
4,Arizona Challenger,Marchal Grande 382,SY005,Pavao Island Marina,Croatia,2016,11.23,3,2,8,423
...,...,...,...,...,...,...,...,...,...,...,...
114,Triple Sycara V,Benedetti 51 Mirage,SY115,Pavao Island Marina,Croatia,2015,15.54,5,3,11,799
115,Tropical Paradise,Neumann 445 Opus,SY116,Pavao Island Marina,Croatia,2013,13.52,4,2,10,640
116,Uptown Girl,Balandro Calypso 37,SY117,Port Djurak,Croatia,2011,11.48,3,1,10,416
117,Vista Mar,Benedetti 44 Alto,SY118,Marina Naupotiri,Greece,2002,13.95,4,2,8,413


In [195]:
bookings_df.head() # Checking the dataframe

,Title,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,Yacht_ID
0,Mr,Finley,Kosar,1580,27 Culmore Cross,Marden,TN21 1ZL,77009001111,12/6/2022,14,SY0182
1,Ms,Caitlin,Anderson,1322,34 Portelet Road,Southwell,KA8 9LJ,77009001122,26/07/2022,8,SY098
2,Mr,Christopher,Washington,1323,108 Chatsworth Way,New Milton,BH25 6GY,77009001133,7/5/2022,10,SY026
3,Mrs,Nicola,Leighton,1330,108 St Maurices Road,Preston,CT3 8BR,77009001152,7/5/2022,14,SY118
4,Mr,David,Ellis,1338,15 Omega Place,Hatfield,ML6 0XP,77009001154,23/06/2022,14,SY094


In [196]:
print(f'Shape of bookings_df before removing values: {bookings_df.shape}') # print statement to check
def check_id(string): # Creating a function to check if the string is in the correct format
    if re.fullmatch(r'[A-Z]{2}\d{3}', string): # Checking if the string is in the correct format
        return 1 # Returning 1 if correct
    else:
        return 0 # Returning 0 if not correct
for index in range(len(bookings_df)): # Iterating through every row in Yacht_ID column
    ids = bookings_df.loc[index, 'Yacht_ID'] # Getting the value 
    result = check_id(ids) # Checking through the function
    if result == 0: # Conditional statement
       bookings_df = bookings_df.drop(index) # If returned 0, row is dropped
bookings_df = bookings_df.reset_index(drop=True) # Resetting the index to have a continuous data
print(f'Shape of bookings_df after removing values that are out of format: {bookings_df.shape}') # Checking the shape to make sure the process worked

Shape of bookings_df before removing values: (979, 11)
Shape of bookings_df after removing values that are out of format: (976, 11)


In [197]:
new_df = pd.DataFrame() # Creating a new dataframe to store the merged dataframe
new_df = pd.merge(yacht_df, bookings_df, left_on='Yacht_Ref', right_on='Yacht_ID') # Merging the dataframe
new_df # Checking the merged dataframe

,Yacht_Name,Yacht_Make,Yacht_Ref,Depart_Port,Depart_Country,Built,Length(m),Cabins,Toilets,Guests,...,First_Name,Last_Name,Cust_ID,Street_Address,Area,Post_Code,Telephone,Depart_Date,Days,Yacht_ID
0,Adriatic Star,Balandro Calypso 45,SY001,Pavao Island Marina,Croatia,2013,13.85,4,2,10,...,Margaret,De Villiers,1430,120 Stone Cellar Road,Boraston,BR16 6ZB,77009001391,7/5/2022,18,SY001
1,Adriatic Star,Balandro Calypso 45,SY001,Pavao Island Marina,Croatia,2013,13.85,4,2,10,...,Myra,Khan,1963,28 Walwyn Road,Chastleton,GL56 3XY,77009002929,29/06/2022,10,SY001
2,Adriatic Star,Balandro Calypso 45,SY001,Pavao Island Marina,Croatia,2013,13.85,4,2,10,...,Jason,Idris,2263,140 Freezeland Lane,Hutingdon,AB38 7WB,77009003955,8/6/2022,7,SY001
3,Adriatic Star,Balandro Calypso 45,SY001,Pavao Island Marina,Croatia,2013,13.85,4,2,10,...,Baek Hyeon,Chong,2440,134 Dorset Walk,Kingsdon,TA11 8YD,77009004673,25/05/2022,14,SY001
4,Adriatic Star,Balandro Calypso 45,SY001,Pavao Island Marina,Croatia,2013,13.85,4,2,10,...,Rachel,Brookes,3167,31 Coley Street,Blackpool Gate,CA6 2RA,77009006699,9/7/2022,21,SY001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,Vivaldi Empress,Marchal Grande 512,SY119,Port Djurak,Croatia,2017,15.20,5,2,12,...,Reece,Hunt,3175,123 Beaumont Avenue,Basingstoke,BS30 0YL,77009006745,7/5/2022,14,SY119
972,Vivaldi Empress,Marchal Grande 512,SY119,Port Djurak,Croatia,2017,15.20,5,2,12,...,Shirley,Gardner,3177,97 Drayton Terrace,Durham,IP22 9PU,77009006761,5/9/2022,14,SY119
973,Vivaldi Empress,Marchal Grande 512,SY119,Port Djurak,Croatia,2017,15.20,5,2,12,...,Muhammad,Shah,3469,142 Hendford Hill,Brandon,HR2 7TZ,77009007672,21/06/2022,10,SY119
974,Vivaldi Empress,Marchal Grande 512,SY119,Port Djurak,Croatia,2017,15.20,5,2,12,...,Fernanda,Wainwright,3896,95 Lowndes Mews,East Tilbury,RM18 0PR,77009008794,17/08/2022,7,SY119


In [198]:
new_df.dtypes # Checking the datatypes of each column

Yacht_Name         object
Yacht_Make         object
Yacht_Ref          object
Depart_Port        object
Depart_Country     object
Built               int64
Length(m)         float64
Cabins              int64
Toilets             int64
Guests              int64
Day_Rate            int64
Title              object
First_Name         object
Last_Name          object
Cust_ID             int64
Street_Address     object
Area               object
Post_Code          object
Telephone           int64
Depart_Date        object
Days                int64
Yacht_ID           object
dtype: object

In [199]:
new_df['Depart_Date'] = pd.to_datetime(new_df['Depart_Date'], dayfirst=True) # Changing the datatype of Depart Date into datetime
new_df['Depart_Date'] = new_df['Depart_Date'].dt.strftime('%d-%m-%Y') # Changing the format of the date
new_df['Telephone'] = new_df['Telephone'].astype(str) # Changing the datatype from int64 into a string as required